In [10]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

class MyLayer(Layer):
    def __init__(self, latent_dim, **kwargs):
        self.latent_dim = latent_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.model = self.build_model()
        super(MyLayer, self).build(input_shape)

    def call(self, inputs):
        return self.build_generator()(inputs)

    def build_model(self):
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        return Model(noise, img)
    
    def build_generator(self):
        model = Sequential()
        model.add(Dense(128 * 16 * 16, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((16, 16, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(3, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))
        return model

def load_data(dataset_path):
    images = []
    for filename in os.listdir(dataset_path):
        img = Image.open(os.path.join(dataset_path, filename))
        img = img.resize((64, 64))
        img = tf.cast(img, tf.float32) / 127.5 - 1.0
        images.append(img)
    return np.array(images, dtype=np.float32)

# データセットのパスを指定
dataset_path = r"C:\Users\user\Downloads\sample-cards-data\images\images"
X_train = load_data(dataset_path)

img_rows = 64
img_cols = 64
channels = 3
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

def build_discriminator(img_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

# GANの構築
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(latent_dim,))
    fake_img = generator(gan_input)
    gan_output = discriminator(fake_img)
    gan = Model(gan_input, gan_output)
    gan.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    return gan

# 訓練用関数
def train_gan(generator, discriminator, gan, X_train, epochs, batch_size=32, save_interval=10):
    valid = np.ones((batch_size, 1), dtype=tf.float32)
    fake = np.zeros((batch_size, 1), dtype=tf.float32)

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(tf.convert_to_tensor(imgs), valid)
        d_loss_fake = discriminator.train_on_batch(tf.convert_to_tensor(gen_imgs), fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        g_loss = gan.train_on_batch(noise, valid)

        # 進捗の表示
        print(f"{epoch} [D loss: {d_loss[0]}] [G loss: {g_loss}]")

        # 一定間隔で生成画像を保存
        if epoch % save_interval == 0:
            save_imgs(generator, epoch)

# 生成した画像を保存する関数
def save_imgs(generator, epoch, r=5, c=5):
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # 生成した画像を0から1の範囲に変換
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt])
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"gan_generated_image_epoch_{epoch}.png")
    plt.close()

# パラメータ設定
epochs = 1000
batch_size = 32
save_interval = 50

# モデルの構築と訓練
generator = MyLayer(latent_dim)(Input(shape=(latent_dim,)))
discriminator = build_discriminator(img_shape)

discriminator.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
generator.compile(optimizer=optimizer, loss='binary_crossentropy')
gan = build_gan(generator, discriminator)
gan.compile(optimizer=optimizer, loss='binary_crossentropy')

train_gan(generator, discriminator, gan, X_train, epochs=epochs, batch_size=batch_size, save_interval=save_interval)

NameError: name 'model' is not defined

In [16]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

# データの読み込み
def load_data(dataset_path):
    images = []
    for filename in os.listdir(dataset_path):
        img = Image.open(os.path.join(dataset_path, filename))
        img = img.resize((64, 64))
        img = tf.cast(img, tf.float32) / 127.5 - 1.0
        images.append(img)
    return np.array(images, dtype=np.float32)

# データセットのパスを指定
dataset_path = r"C:\Users\user\Downloads\sample-cards-data\images\images"
X_train = load_data(dataset_path)

# GANのパラメータ設定
img_rows = 64
img_cols = 64
channels = 3
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

class MyLayer(Layer):
    def __init__(self, latent_dim, **kwargs):
        self.latent_dim = latent_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.model = self.build_model()
        super(MyLayer, self).build(input_shape)

    def call(self, inputs):
        img = self.model(inputs)
        return img

    def build_model(self):
        noise = Input(shape=(self.latent_dim,))
        x = Dense(128 * 16 * 16, activation="relu")(noise)
        x = Reshape((16, 16, 128))(x)
        x = UpSampling2D()(x)
        x = Conv2D(128, kernel_size=3, padding="same")(x)
        x = BatchNormalization(momentum=0.8)(x)
        x = Activation("relu")(x)
        x = UpSampling2D()(x)
        x = Conv2D(64, kernel_size=3, padding="same")(x)
        x = BatchNormalization(momentum=0.8)(x)
        x = Activation("relu")(x)
        x = Conv2D(3, kernel_size=3, padding="same")(x)
        outputs = Activation("tanh")(x)
        return Model(noise, outputs)

def build_generator(latent_dim, img_shape):
    inputs = Input(shape=(latent_dim,))
    x = MyLayer(latent_dim)(inputs)
    model = Model(inputs, x)
    return model

# 識別器の構築
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

# GANの構築
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(latent_dim,))
    fake_img = generator(gan_input)
    gan_output = discriminator(fake_img)
    gan = Model(gan_input, gan_output)
    gan.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    return gan

# 訓練用関数
def train_gan(generator, discriminator, gan, X_train, epochs, batch_size=32, save_interval=10):
    valid = np.ones((batch_size, 1), dtype=tf.float32)
    fake = np.zeros((batch_size, 1), dtype=tf.float32)

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(tf.convert_to_tensor(imgs), valid)
        d_loss_fake = discriminator.train_on_batch(tf.convert_to_tensor(gen_imgs), fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        g_loss = gan.train_on_batch(noise, valid)

        # 進捗の表示
        print(f"{epoch} [D loss: {d_loss[0]}] [G loss: {g_loss}]")

        # 一定間隔で生成画像を保存
        if epoch % save_interval == 0:
            save_imgs(generator, epoch)

# 生成した画像を保存する関数
def save_imgs(generator, epoch, r=5, c=5):
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # 生成した画像を0から1の範囲に変換
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt])
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"gan_generated_image_epoch_{epoch}.png")
    plt.close()

# パラメータ設定
epochs = 1000
batch_size = 32
save_interval = 50

# モデルの構築と訓練
discriminator = build_discriminator(img_shape)

discriminator.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
gan = build_gan(generator, discriminator)
gan.compile(optimizer=optimizer, loss='binary_crossentropy')

train_gan(generator, discriminator, gan, X_train, epochs=epochs, batch_size=batch_size, save_interval=save_interval)

TypeError: 'KerasTensor' object is not callable

In [23]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

class MyLayer(Layer):
    def __init__(self, latent_dim, **kwargs):
        self.latent_dim = latent_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.generator = self.build_generator()
        super(MyLayer, self).build(input_shape)

    def call(self, inputs):
        img = self.generator(inputs)
        return img

    def build_generator(self):
        noise = Input(shape=(self.latent_dim,))
        x = Dense(128 * 16 * 16, activation="relu")(noise)
        x = Reshape((16, 16, 128))(x)
        x = UpSampling2D()(x)
        x = Conv2D(128, kernel_size=3, padding="same")(x)
        x = BatchNormalization(momentum=0.8)(x)
        x = Activation("relu")(x)
        x = UpSampling2D()(x)
        x = Conv2D(64, kernel_size=3, padding="same")(x)
        x = BatchNormalization(momentum=0.8)(x)
        x = Activation("relu")(x)
        x = Conv2D(3, kernel_size=3, padding="same")(x)
        outputs = Activation("tanh")(x)
        return Model(noise, outputs)
        
# データの読み込み
def load_data(dataset_path):
    images = []
    for filename in os.listdir(dataset_path):
        img = Image.open(os.path.join(dataset_path, filename))
        img = img.resize((64, 64))
        img = tf.cast(img, tf.float32) / 127.5 - 1.0
        images.append(img)
    return np.array(images, dtype=np.float32)

# データセットのパスを指定
dataset_path = r"C:\Users\user\Downloads\sample-cards-data\images\images"
X_train = load_data(dataset_path)

# GANのパラメータ設定
img_rows = 64
img_cols = 64
channels = 3
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

def build_generator(latent_dim, img_shape):
    inputs = Input(shape=(latent_dim,))
    x = MyLayer(latent_dim)(inputs)
    model = Model(inputs, x)
    return model

# 識別器の構築
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

# GANの構築
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(latent_dim,))
    fake_img = generator(gan_input)
    gan_output = discriminator(fake_img)
    gan = Model(gan_input, gan_output)
    gan.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    return gan

# 訓練用関数
def train_gan(generator, discriminator, gan, X_train, epochs, batch_size=32, save_interval=10):
    valid = np.ones((batch_size, 1), dtype=np.float32)
    fake = np.zeros((batch_size, 1), dtype=np.float32)

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        g_loss = gan.train_on_batch(noise, valid)

        # 進捗の表示
        print(f"{epoch} [D loss: {d_loss[0]}] [G loss: {g_loss}]")

        # 一定間隔で生成画像を保存
        if epoch % save_interval == 0:
            save_imgs(generator, epoch)

# 生成した画像を保存する関数
def save_imgs(generator, epoch, r=5, c=5):
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # 生成した画像を0から1の範囲に変換
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt])
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"gan_generated_image_epoch_{epoch}.png")
    plt.close()

# パラメータ設定
epochs = 1000
batch_size = 32
save_interval = 50

# モデルの構築と訓練
generator = build_generator(latent_dim, img_shape)
discriminator = build_discriminator(img_shape)

discriminator.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
generator.compile(optimizer=optimizer, loss='binary_crossentropy')
gan = build_gan(generator, discriminator)
gan.compile(optimizer=optimizer, loss='binary_crossentropy')

train_gan(generator, discriminator, gan, X_train, epochs=epochs, batch_size=batch_size, save_interval=save_interval)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 572ms/step


ValueError: Input 0 of layer "functional_87" is incompatible with the layer: expected shape=(None, 64, 64, 3), found shape=(32, 64, 64, 4)

In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

class MyLayer(Layer):
    def __init__(self, latent_dim, **kwargs):
        self.latent_dim = latent_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.generator = self.build_generator()
        super(MyLayer, self).build(input_shape)

    def call(self, inputs):
        img = self.generator(inputs)
        return img

    def build_generator(self):
        noise = Input(shape=(self.latent_dim,))
        x = Dense(128 * 16 * 16, activation="relu")(noise)
        x = Reshape((16, 16, 128))(x)
        x = UpSampling2D()(x)
        x = Conv2D(128, kernel_size=3, padding="same")(x)
        x = BatchNormalization(momentum=0.8)(x)
        x = Activation("relu")(x)
        x = UpSampling2D()(x)
        x = Conv2D(64, kernel_size=3, padding="same")(x)
        x = BatchNormalization(momentum=0.8)(x)
        x = Activation("relu")(x)
        x = Conv2D(3, kernel_size=3, padding="same")(x)
        outputs = Activation("tanh")(x)
        return Model(noise, outputs)

# データの読み込み
def load_data(dataset_path):
    images = []
    for filename in os.listdir(dataset_path):
        img = Image.open(os.path.join(dataset_path, filename)).convert('RGB')  # ここでRGBに変換
        img = img.resize((64, 64))
        img = np.array(img)
        img = (img / 127.5) - 1.0
        images.append(img)
    return np.array(images, dtype=np.float32)

# データセットのパスを指定
dataset_path = r"C:\Users\user\Downloads\sample-cards-data\images\images"
X_train = load_data(dataset_path)

# GANのパラメータ設定
img_rows = 64
img_cols = 64
channels = 3
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

def build_generator(latent_dim, img_shape):
    inputs = Input(shape=(latent_dim,))
    x = MyLayer(latent_dim)(inputs)
    model = Model(inputs, x)
    return model

# 識別器の構築
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

# GANの構築
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(latent_dim,))
    fake_img = generator(gan_input)
    gan_output = discriminator(fake_img)
    gan = Model(gan_input, gan_output)
    gan.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    return gan

# 訓練用関数
def train_gan(generator, discriminator, gan, X_train, epochs, batch_size=32, save_interval=10):
    valid = np.ones((batch_size, 1), dtype=np.float32)
    fake = np.zeros((batch_size, 1), dtype=np.float32)

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        g_loss = gan.train_on_batch(noise, valid)

        # 進捗の表示
        print(f"{epoch} [D loss: {d_loss[0]}] [G loss: {g_loss}]")

        # 一定間隔で生成画像を保存
        if epoch % save_interval == 0:
            save_imgs(generator, epoch)

# 生成した画像を保存する関数
def save_imgs(generator, epoch, r=5, c=5):
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # 生成した画像を0から1の範囲に変換
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt])
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"gan_generated_image_epoch_{epoch}.png")
    plt.close()

# パラメータ設定
epochs = 1000
batch_size = 32
save_interval = 50

# モデルの構築と訓練
generator = build_generator(latent_dim, img_shape)
discriminator = build_discriminator(img_shape)

discriminator.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
generator.compile(optimizer=optimizer, loss='binary_crossentropy')
gan = build_gan(generator, discriminator)
gan.compile(optimizer=optimizer, loss='binary_crossentropy')

train_gan(generator, discriminator, gan, X_train, epochs=epochs, batch_size=batch_size, save_interval=save_interval)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step


C:\Users\user\anaconda3\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:71: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0 [D loss: 0.7263780832290649] [G loss: [array(0.7145185, dtype=float32), array(0.7145185, dtype=float32), array(0.7145185, dtype=float32), array(0.515625, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1 [D loss: 0.7195343375205994] [G loss: [array(0.7153421, dtype=float32), array(0.7153421, dtype=float32), array(0.7153421, dtype=float32), array(0.515625, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
2 [D loss: 0.7170929312705994] [G loss: [array(0.7146282, dtype=float32), array(0.7146282, dtype=float32), array(0.7146282, dtype=float32), array(0.515625, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
3 [D loss: 0.7140219807624817] [G loss: [array(0.7123029, dtype=float32), array(0.7123029, dtype=float32), array(0.7123029, dtype=float32), array(0.52734375, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
4 [D loss: 0.712951123714447] [G loss: [array(0.71164405, dtype=float32), array(0.71164405, dtype=float32), arr